In [ ]:
%pip install "torch==2.2.2" tensorboard
!pip install numpy==1.26.4
# Install Hugging Face libraries
%pip install  --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0"

  Using cached transformers-4.40.0-py3-none-any.whl.metadata (137 kB)
  Using cached datasets-2.18.0-py3-none-any.whl.metadata (20 kB)
  Using cached accelerate-0.29.3-py3-none-any.whl.metadata (18 kB)
  Using cached evaluate-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl.metadata (2.2 kB)
  Using cached trl-0.8.6-py3-none-any.whl.metadata (11 kB)
  Using cached peft-0.10.0-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.40.0-py3-none-any.whl (9.0 MB)
Using cached datasets-2.18.0-py3-none-any.whl (510 kB)
Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
Using cached evaluate-0.4.1-py3-none-any.whl (84 kB)
Using cached bitsandbytes-0.43.1-py3-none-manylinux_2_24_x86_64.whl (119.8 MB)
Using cached trl-0.8.6-py3-none-any.whl (245 kB)
Using cached peft-0.10.0-py3-none-any.whl (199 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      

In [ ]:
!pip install numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 93.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.16.0 requires transformers, which is not installed.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, which is not installed.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
torchvision 0.21.0+cu124 requires torch==2.6.0, but you have torch 2.2.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
peft 0.16.0 requi

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).

In [ ]:
import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Adjust the path as needed for your Google Drive folder structure
file_path = '/content/drive/My Drive/Thesis files/sampled_EventRelevance_1_df.parquet'
df = pd.read_parquet(file_path)

# Preview to verify

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
0# Mapping values
mapping = {1: 'shortage', -1: 'surplus', 0: 'stagnant'}

# Create the new column
df['label'] = df['price'].map(mapping)

df

,TIMESTAMP_UTC,RP_DOCUMENT_ID,EVENT_SENTIMENT,EVENT_RELEVANCE,TITLE,EVENT_TEXT,SOURCE_RANK,CATEGORY,price,YEAR,label
0,2000-05-17 00:00:04.208,83B221600F7B3F19F1AC0545334EC153,-0.86,100,Economic Indicators - Oil production surges 54...,Oil production surges 54% in first quarter,3,supply-increase-commodity,-1,2000,surplus
1,2000-09-11 00:00:09.687,EE850A4CB3F4EADC317A10390E4FF4F1,-0.16,100,ONAKO - Oil production up 0.1% for 8 months to...,Oil production up 0.1% for 8 months,2,supply-increase-commodity,-1,2000,surplus
2,2000-08-09 00:00:07.774,D9FFFFE8063D0790F62681A270462702,-0.16,100,Udmurtia - oil production up 0.1% to 3.8mln to...,Oil production in Udmurtia grew 0.1% year-on-y...,2,supply-increase-commodity,-1,2000,surplus
3,2000-05-24 00:00:04.552,AA428067DE0BA7F610CAFEB50E14A570,0.64,100,"JVs' oil output falls in April by 18,600 t to ...",Jvs' oil output falls in April,2,supply-decrease-commodity,1,2000,shortage
4,2000-05-17 00:00:05.399,FE606B5A5C5744533F0434FF7A1CAFE5,0.63,100,Rising prices curb Thai oil demand,Oil demand is expected to pick up marginally,2,demand-increase-commodity,1,2000,shortage
...,...,...,...,...,...,...,...,...,...,...,...
5195,2025-02-15 11:24:07.646,2D5981B32BC0185792ED20C40AFC0BCB,-0.54,100,Russia Renews Its Failed Promises to Increase ...,To Increase Oil Production in Jaruco,3,supply-increase-commodity,-1,2025,surplus
5196,2025-01-03 16:13:57.832,E3D691C8BEF4F2FB9F908C26ED92B805,-0.86,100,PDVSA 2024 Oil Exports Climb in Choppy Year,Oil exports to the U.S. soared 64% to some 222...,3,exports-up,-1,2025,surplus
5197,2025-05-01 12:35:18.782,00B70AEA70CBB6D1071245C0AE42442A,-0.52,100,Kuwait's March crude oil exports to Japan up 1...,Kuwait's crude oil exports to Japan in March g...,4,exports-up,-1,2025,surplus
5198,2025-02-12 13:29:08.555,702138F756CBF549E4BC7F3F9D2DCB3B,0.44,100,Russia's oil output edges down 0.3% in January...,Russia's oil output edges down 0.3% in January,3,supply-decrease-commodity,1,2025,shortage


In [ ]:
df['label']=df['label'].astype('category')
df['target']=df['label'].cat.codes

In [ ]:
category_map = {code: category for code, category in enumerate(df['label'].cat.categories)}
category_map

{0: 'shortage', 1: 'stagnant', 2: 'surplus'}

In [ ]:
train_end_point = int(df.shape[0]*0.6)
val_end_point = int(df.shape[0]*0.8)
df_train = df.iloc[:train_end_point,:]
df_val = df.iloc[train_end_point:val_end_point,:]
df_test = df.iloc[val_end_point:,:]
print(df_train.shape, df_test.shape, df_val.shape)

(3120, 12) (1040, 12) (1040, 12)


In [ ]:
# Converting pandas DataFrames into Hugging Face Dataset objects:
dataset_train = Dataset.from_pandas(df_train.drop('label',axis=1))
dataset_val = Dataset.from_pandas(df_val.drop('label',axis=1))
dataset_test = Dataset.from_pandas(df_test.drop('label',axis=1))

In [ ]:
dataset_train_shuffled = dataset_train.shuffle(seed=42)  # Using a seed for reproducibility


In [ ]:
# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train_shuffled,
    'val': dataset_val,
    'test': dataset_test
})
dataset

DatasetDict({
    train: Dataset({
        features: ['TIMESTAMP_UTC', 'RP_DOCUMENT_ID', 'EVENT_SENTIMENT', 'EVENT_RELEVANCE', 'TITLE', 'EVENT_TEXT', 'SOURCE_RANK', 'CATEGORY', 'price', 'YEAR', 'target'],
        num_rows: 3120
    })
    val: Dataset({
        features: ['TIMESTAMP_UTC', 'RP_DOCUMENT_ID', 'EVENT_SENTIMENT', 'EVENT_RELEVANCE', 'TITLE', 'EVENT_TEXT', 'SOURCE_RANK', 'CATEGORY', 'price', 'YEAR', 'target'],
        num_rows: 1040
    })
    test: Dataset({
        features: ['TIMESTAMP_UTC', 'RP_DOCUMENT_ID', 'EVENT_SENTIMENT', 'EVENT_RELEVANCE', 'TITLE', 'EVENT_TEXT', 'SOURCE_RANK', 'CATEGORY', 'price', 'YEAR', 'target'],
        num_rows: 1040
    })
})

In [ ]:
df_train.target.value_counts(normalize=True)


,proportion
target,
0,0.461859
2,0.441667
1,0.096474


In [ ]:
class_weights=(1/df_train.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights


tensor([0.1463, 0.7006, 0.1530])

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"


In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

In [ ]:
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=3
)

model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [ ]:
model = prepare_model_for_kbit_training(model)
model

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [ ]:
model = get_peft_model(model, lora_config)
model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): LlamaForSequenceClassification(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
        

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
sentences = df_test.TITLE.tolist()
sentences[0:2]

['OPEC, Corona19 Forecast Global Oil Demand Decline This Year... Down By 100,000 Barrels',
 'OPEC ministers agree to cut oil production due to COVID-19']

In [ ]:
# Convert summaries to a list
sentences = df_test.TITLE.tolist()

# Define the batch size
batch_size = 32  # You can adjust this based on your system's memory capacity

# Initialize an empty list to store the model outputs
all_outputs = []

# Process the sentences in batches
for i in range(0, len(sentences), batch_size):
    # Get the batch of sentences
    batch_sentences = sentences[i:i + batch_size]

    # Tokenize the batch
    inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move tensors to the device where the model is (e.g., GPU or CPU)
    inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

    # Perform inference and store the logits
    with torch.no_grad():
        outputs = model(**inputs)
        all_outputs.append(outputs['logits'])

In [ ]:
final_outputs = torch.cat(all_outputs, dim=0)
final_outputs

tensor([[-5.3222, -1.7052,  4.0408],
        [-3.1264,  0.3517,  0.1961],
        [-2.7881, -0.1275,  2.2525],
        ...,
        [-3.3315, -2.5682,  6.0065],
        [-3.7423, -0.1506,  2.5018],
        [ 0.1845, -2.1194,  4.8240]], device='cuda:0')

In [ ]:
final_outputs.argmax(axis=1)


tensor([2, 1, 2,  ..., 2, 2, 2], device='cuda:0')

In [ ]:
df_test['predictions'] = final_outputs.argmax(axis=1).cpu().tolist()


/tmp/ipython-input-27-1844382488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions'] = final_outputs.argmax(axis=1).cpu().tolist()


In [ ]:
df_test['predictions'].value_counts()


,count
predictions,
2,857
1,156
0,27


In [ ]:
df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])
df_test['predictions']

/tmp/ipython-input-29-4293232383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


,predictions
4160,surplus
4161,stagnant
4162,surplus
4163,surplus
4164,surplus
...,...
5195,surplus
5196,surplus
5197,surplus
5198,surplus


In [ ]:
def get_performance_metrics(df_test):
  y_test = df_test.label
  y_pred = df_test.predictions

  print("Confusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

  print("\nClassification Report:")
  print(classification_report(y_test, y_pred))

  print("Balanced Accuracy Score:", balanced_accuracy_score(y_test, y_pred))
  print("Accuracy Score:", accuracy_score(y_test, y_pred))

In [ ]:
get_performance_metrics(df_test)


Confusion Matrix:
[[ 20 117 397]
 [  1   5 115]
 [  6  34 345]]

Classification Report:
              precision    recall  f1-score   support

    shortage       0.74      0.04      0.07       534
    stagnant       0.03      0.04      0.04       121
     surplus       0.40      0.90      0.56       385

    accuracy                           0.36      1040
   macro avg       0.39      0.32      0.22      1040
weighted avg       0.53      0.36      0.25      1040

Balanced Accuracy Score: 0.3249597978913607
Accuracy Score: 0.3557692307692308


In [ ]:
MAX_LEN = 512
col_to_delete = ['TIMESTAMP_UTC', 'RP_DOCUMENT_ID', 'EVENT_SENTIMENT', 'EVENT_RELEVANCE', 'TITLE', 'EVENT_TEXT', 'SOURCE_RANK', 'CATEGORY', 'price', 'YEAR']


def llama_preprocessing_function(examples):
    return tokenizer(examples['TITLE'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "label")

Map:   0%|          | 0/3120 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

Map:   0%|          | 0/1040 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets['train'][0]

{'label': 2,
 'input_ids': [128000,
  53,
  21198,
  24669,
  79118,
  311,
  709,
  5707,
  13086,
  311,
  5734],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
collate_fn = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}

In [ ]:
class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir = 'sentiment_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 3,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True
)


In [ ]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['val'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

/tmp/ipython-input-36-2851958097.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)


In [ ]:
train_result = trainer.train()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: adamgurdikyan2001 (adamgurdikyan2001-queen-mary-university-of-london) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Balanced Accuracy,Accuracy
1,No log,0.262691,0.904214,0.906731
2,No log,0.190987,0.935079,0.944231
3,0.369100,0.225166,0.950216,0.950962


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
def make_predictions(model,df_test):


  # Convert summaries to a list
  sentences = df_test.TITLE.tolist()

  # Define the batch size
  batch_size = 32  # You can adjust this based on your system's memory capacity

  # Initialize an empty list to store the model outputs
  all_outputs = []

  # Process the sentences in batches
  for i in range(0, len(sentences), batch_size):
      # Get the batch of sentences
      batch_sentences = sentences[i:i + batch_size]

      # Tokenize the batch
      inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          all_outputs.append(outputs['logits'])
  final_outputs = torch.cat(all_outputs, dim=0)
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


make_predictions(model,df_test)

/tmp/ipython-input-40-354763479.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
/tmp/ipython-input-40-354763479.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predictions']=df_test['predictions'].apply(lambda l:category_map[l])


In [ ]:
get_performance_metrics(df_test)


Confusion Matrix:
[[503   9  22]
 [  1 119   1]
 [  8   3 374]]

Classification Report:
              precision    recall  f1-score   support

    shortage       0.98      0.94      0.96       534
    stagnant       0.91      0.98      0.94       121
     surplus       0.94      0.97      0.96       385

    accuracy                           0.96      1040
   macro avg       0.94      0.97      0.95      1040
weighted avg       0.96      0.96      0.96      1040

Balanced Accuracy Score: 0.9656157371172692
Accuracy Score: 0.9576923076923077
